In [11]:
import sys 
import csv
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
##import sexmachine.detector as gender
from sklearn.preprocessing import Imputer
from sklearn import cross_validation 
from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, train_test_split 
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.learning_curve import learning_curve
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
%matplotlib inline

/Users/farooqkhadim/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/farooqkhadim/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/farooqkhadim/anaconda/lib/python3.6/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will 

In [33]:
 def read_datasets():
    genuine_users = pd.read_csv("data/not_fake_comments.csv")
    fake_users = pd.read_csv("data/fake_comments.csv") 
    # print genuine_users.columns
    # print genuine_users.describe()
    #print fake_users.describe()
    x=pd.concat([genuine_users,fake_users])
    y=len(fake_users)*[0] + len(genuine_users)*[1] 
    return x,y

In [4]:
def extract_features(x):
    lang_list = list(enumerate(np.unique(x['lang'])))
    lang_dict = { name : i for i, name in lang_list }
    x.loc[:,'lang_code'] = x['lang'].map( lambda x: lang_dict[x]).astype(int)
    feature_columns_to_use = ['statuses_count','followers_count','friend s_count','favourites_count','listed_count','lang_code']
    x=x.loc[:,feature_columns_to_use] 
    return x

In [12]:
def plot_learning_curve(estimator,title,X,y,ylim=None,cv=None, n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes) 
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1) 
    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1,color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
    label="Cross-validation score") 
    plt.legend(loc="best")
    return plt


In [15]:
def plot_confusion_matrix(cm,title='Confusionmatrix',cmap=plt.cm.Blues):
    target_names=['Fake','Genuine']
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names) 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [19]:
def plot_roc_curve(y_test,y_pred):
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    print ("False Positive rate: ",false_positive_rate)
    print ("True Positive rate: ",true_positive_rate)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right') 
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([-0.1,1.2])
    plt.ylim([-0.1,1.2])
    plt.ylabel('True Positive Rate') 
    plt.xlabel('False Positive Rate') 
    plt.show()

In [25]:
def train(X_train,y_train,X_test):
    """ Trains and predicts dataset with a SVM classifier """ # Scaling features
    X_train=preprocessing.scale(X_train) 
    X_test=preprocessing.scale(X_test)
    Cs = 10.0 ** np.arange(-2,3,.5)
    gammas = 10.0 ** np.arange(-2,3,.5)
    param = [{'gamma': gammas, 'C': Cs}]
    cvk = StratifiedKFold(y_train,n_folds=5)
    classifier = SVC()
    clf = GridSearchCV(classifier,param_grid=param,cv=cvk)
    clf.fit(X_train,y_train)
    print("The best classifier is: ",clf.best_estimator_) 
    clf.best_estimator_.fit(X_train,y_train)
    # Estimate score
    scores = cross_validation.cross_val_score(clf.best_estimator_, X_train,y_train, cv=5)
    print (scores)
    print('Estimated score: %0.5f (+/- %0.5f)' % (scores.mean(), scores.std() / 2))
    title = 'Learning Curves (SVM, rbf kernel, $\gamma=%.6f$)' %clf.best_estimator_.gamma
    plot_learning_curve(clf.best_estimator_, title, X_train, y_train, cv=5)
    plt.show()
    # Predict class
    y_pred = clf.best_estimator_.predict(X_test) 
    return y_test,y_pred
 

In [32]:
print ("readingdatasets.....\n")
x,y=read_datasets()

readingdatasets.....



UnicodeDecodeError: 'utf-8' codec can't decode byte 0x83 in position 111: invalid start byte

In [30]:
print("extractingfeatues.....\n") 
x=extract_features(x)
print (x.columns)
print (x.describe())

extractingfeatues.....



NameError: name 'x' is not defined

In [ ]:
print("splitingdatasetsintrainandtestdataset...\n") 
X_train,X_test,y_train,y_test = train_test_split(x, y, test_size=0.20, ran dom_state=44)

In [ ]:
print("trainingdatasets.......\n")
y_test,y_pred = train(X_train,y_train,X_test)

In [ ]:
print('ClassificationAccuracyonTestdataset:',accuracy_score(y_test, y_pred))

In [ ]:
cm=confusion_matrix(y_test,y_pred) 
print('Confusion matrix, without normalization')
print(cm)
plot_confusion_matrix(cm)

In [ ]:
cm_normalized=cm.astype('float')/cm.sum(axis=1)[:,np.newaxis] 
print('Normalized confusion matrix')
print(cm_normalized)
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

In [ ]:
print(classification_report(y_test,y_pred,target_names=['Fake','Genuin e']))

In [ ]:
plot_roc_curve(y_test,y_pred)